# Backtest a custom momentum strategy with Zipline

Zipline is a Python library that allows you to backtest trading algorithms using historical data.

It was developed by Quantopian in 2012 and used to manage their $100,000,000 crowdfunded hedge fund.

In [1]:
import pandas as pd

from zipline import run_algorithm
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.data import USEquityPricing
from zipline.api import (
    attach_pipeline,
    calendars,
    pipeline_output,
    date_rules,
    time_rules,
    set_commission,
    set_slippage,
    order_target_percent,
    get_open_orders,
    schedule_function
)

import warnings
warnings.filterwarnings("ignore")

N_LONGS = 10

In [11]:
# %load_ext zipline # Load the zipline extension
! zipline ingest -b quandl

[2023-07-05T08:04:26-0500-INFO][zipline.data.bundles.core]
 Ingesting quandl
[2023-07-05T08:04:26-0500-INFO][zipline.data.bundles.quandl]
[2023-07-05T08:04:35-0500-INFO][zipline.data.bundles.quandl]
 Parsing raw data.
[2023-07-05T08:04:48-0500-INFO][zipline.data.bundles.quandl]
 Generating asset metadata.
Merging daily equity files:  [#-----------------------------------]  1731/opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.9/site-packages/zipline/data/bcolz_daily_bars.py:341: UserWarning: Ignoring 1 values because they are out of bounds for uint32:
             open  high   low  close        volume  ex_dividend  split_ratio
2011-04-11  1.79  1.84  1.55    1.7  6.674913e+09          0.0          1.0
  winsorise_uint32(raw_data, invalid_data_behavior, "volume", *OHLC)
Merging daily equity files:  [####################################]      
[2023-07-05T08:05:31-0500-INFO][zipline.data.bundles.quandl]
 Parsing split data.
[2023-07-05T08:05:31-0500-INFO][zipline.data.bundles

In [6]:
# Create a custom momentum factor and build the pipeline

# Now, create a custom momentum to calculate the momentum and define the pipeline for our trading algorithm.

class Momentum(CustomFactor):
    # Default inputs
    inputs = [USEquityPricing.close]

    # Compute momentum
    def compute(self, today, assets, out, close):
        out[:] = close[-1] / close[0]


def make_pipeline():

    twenty_day_momentum = Momentum(window_length=20)
    thirty_day_momentum = Momentum(window_length=30)

    positive_momentum = (
        (twenty_day_momentum > 1) and
        (thirty_day_momentum > 1)
    )

    return Pipeline(
        columns={
            'longs': thirty_day_momentum.top(N_LONGS),
        },
        screen=positive_momentum
    )

In [7]:
# Implement the trading strategy

# Implement the trading strategy by initializing the pipeline, scheduling the rebalancing function, and executing the trades.

def before_trading_start(context, data):
    context.factor_data = pipeline_output("factor_pipeline")
    assets = context.factor_data.index

def initialize(context):
    attach_pipeline(make_pipeline(), "factor_pipeline")
    schedule_function(
        rebalance,
        date_rules.week_start(),
        time_rules.market_open(),
        calendar=calendars.US_EQUITIES,
    )

def rebalance(context, data):
    factor_data = context.factor_data
    assets = factor_data.index

    longs = assets[factor_data.longs]
    divest = context.portfolio.positions.keys() - longs

    exec_trades(data, assets=divest, target_percent=0)
    exec_trades(data, assets=longs, target_percent=1 / N_LONGS if N_LONGS else 0)

def exec_trades(data, assets, target_percent):
    # Loop through every asset...
    for asset in assets:
        # ...if the asset is tradeable and there are no open orders...
        if data.can_trade(asset) and not get_open_orders(asset):
            # ...execute the order against the target percent
            order_target_percent(asset, target_percent)

In [20]:
# Run the backtest
from datetime import datetime
import pytz
start = pd.Timestamp('2015', tz='utc')
end = pd.Timestamp('2018', tz='utc')

# start = pd.Timestamp('2015')
# end = pd.Timestamp('2018')

# start = datetime(2015, 1, 2)
# end = datetime(2018, 12, 30)

perf = run_algorithm(
    start=start,
    end=end,
    initialize=initialize,
    before_trading_start=before_trading_start,
    capital_base=100_000,
    bundle="quandl",
)

AttributeError: 'datetime.timezone' object has no attribute 'zone'

In [14]:
start


Timestamp('2015-01-01 00:00:00+0000', tz='UTC')